In [1]:
import os
from dataloader import DataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from model import unet
from utils import show_history

In [4]:
# input image dimensions
params = {'batch_size': 1,
          'shape': (128, 128, 128),
          'n_channels': 1,
          'shuffle': True}
train_path = "./data/train/"
val_path = "./data/validation/"

train_ids = range(len(os.listdir(train_path)))
val_ids = range(len(os.listdir(val_path)))
train_generator = DataGenerator(path=train_path,
                                ids=train_ids, **params)
valid_generator = DataGenerator(path=val_path,
                                ids=val_ids, **params)
model = unet(input_size=(None, None, None, 1))
model.compile(optimizer=Adam(lr=1e-4), loss='bce',
              metrics=['accuracy'])
model.summary()

# checkpoint
filepath = "model/model-{epoch:02d}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',
                             verbose=1, save_best_only=False, mode='min')
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                              patience=20, min_lr=1e-8)
callbacks_list = [checkpoint]
print("data prepared, ready to train!")
# Fit the model
history = model.fit(train_generator,
                    validation_data=valid_generator, epochs=100, callbacks=callbacks_list, verbose=1)
model.save('model/model.h5')
show_history(history)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 conv3d_15 (Conv3D)             (None, None, None,   448         ['input_2[0][0]']                
                                None, 16)                                                         
                                                                                                  
 conv3d_16 (Conv3D)             (None, None, None,   6928        ['conv3d_15[0][0]']              
                                None, 16)                                                   

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[2,32,128,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_1/concatenate_5/concat-0-TransposeNDHWCToNCDHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_3109]